# ANALYSIS OF Mumbai

## Introduction

The project is intended to submit to Coursera-IBM as a Final Capstone project

In this data analysis exercise, we are going to analyse the city of Mumbai, Maharashtra. It is basically for all the tourist to come to visit India. Mumbai is located in the western part of India and is known as the Island City . It has many places to explore and is one of the important tourist destinations of the country. Hence, analysing the data of Mumbai using foursquare api will be highly helpful for new people who are migrating to Mumbai or comming to visit the city for a short span of time. Mumbai is also famous for being the Heart of Bollywood.

## Problem Statement


Mumbai usually sees more than a lakh of visitors or tourists per year. Mumbai is not a small city but rather a world itself. A person who is visiting the place for the first time might not be much aware about the places or maybe trouble trying to hunt for them. 
So this is just an attempt to help the tourists with good resturants , hotels along with nearby medical shopes,hospitals and a few places to visit.

## Target Audience 

The people who visit Mumbai for holidays or work purpose, or those who are planning to start a new life in Mumbai.

## K-Means - Algorithm

Why

         ▪ The K-means clustering algorithm is used to find groups which have not been
            explicitly labelled in the data. This can be used to confirm business assumptions
            about what types of groups exist or to identify unknown groups in complex data
            sets

        ▪ As we are using data source which has to clustered using unlabelled data, kmeans is a better algorithm as fit- to  purpose

## Importing Libraries

In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
!pip install geopy
from geopy.geocoders import Nominatim 
print ("Importing Libraries Completed !")

Importing Libraries Completed !


In [2]:
import numpy as np 
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Imported")

Imported


In [3]:
!pip install folium 
import folium

## Getting coordinates of Chennai and displaying the map for reference using folium 

In [4]:
address = 'Mumbai'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 19.0759899, 72.8773928.


In [5]:
import folium

mumbai_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
folium.Marker([latitude, longitude]).add_to(mumbai_map)
mumbai_map

## Data

Data for the venues in Mumbai are taken from foursquare api. Foursquare consists of the details of places, their importance, latitudes, longitudes and which category they belong to. Hence, this api would be helpful to analyse the picture of Mumbai. Data is collected using client id and secret created from foursquare website and used for the required analysis.

### Obtaining data from foursquare api

In [6]:
CLIENT_ID = 'YD2WPTVK0HM53V44CXA4CAMOVY5V0YTSKYXO32UFBXS02JKX' # your Foursquare ID
CLIENT_SECRET = 'CULFDRVS4USPQMBS4FJCUDTSNL2BSGRGYUSNRM0AYAMCNRRM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: YD2WPTVK0HM53V44CXA4CAMOVY5V0YTSKYXO32UFBXS02JKX
CLIENT_SECRET:CULFDRVS4USPQMBS4FJCUDTSNL2BSGRGYUSNRM0AYAMCNRRM


In [7]:
LIMIT = 80 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YD2WPTVK0HM53V44CXA4CAMOVY5V0YTSKYXO32UFBXS02JKX&client_secret=CULFDRVS4USPQMBS4FJCUDTSNL2BSGRGYUSNRM0AYAMCNRRM&v=20180605&ll=19.0759899,72.8773928&radius=5000&limit=80'

## Methodology 

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f15b1b2bd9c5b4d00fd3568'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Mumbai',
  'headerFullLocation': 'Mumbai',
  'headerLocationGranularity': 'city',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 19.120989945000044,
    'lng': 72.92491866226877},
   'sw': {'lat': 19.030989854999955, 'lng': 72.82986693773123}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9b39ed2ae860fc0f5a81cb',
       'name': 'Sofitel Mumbai BKC',
       'location': {'address': 'C 57 Bandra Kurla Complex, Bandra East, Maharashtra',
        'crossStreet': 'Bandra East',
        'lat': 19.0674478,
        'lng': 72.8690057,
        'labeledLatLngs': [{'l

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Converting data in json format to a dataframe for further analysis

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


(80, 4)

In [11]:
nearby_venues.head()

,name,categories,lat,lng
0,Sofitel Mumbai BKC,Hotel,19.067448,72.869006
1,Hitchki,Bar,19.069730,72.869761
2,PVR Cinemas,Movie Theater,19.086643,72.889839
3,Starbucks Coffee: A Tata Alliance,Coffee Shop,19.069457,72.869375
4,Hamleys,Toy / Game Store,19.086655,72.889783


## Plotting the map of these venues

In [12]:
mumbai_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

for name, latitude, longitude in zip(nearby_venues['name'], nearby_venues['lat'], nearby_venues['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(mumbai_map)  

mumbai_map

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'categories', 
                  'lat', 
                  'lng', 
                ]
    
    return(nearby_venues)

### Getting the names of the venues listed in map

In [14]:
mumbai_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

Sofitel Mumbai BKC
Hitchki
PVR Cinemas
Starbucks Coffee: A Tata Alliance
Hamleys
Trident
The Bar Stock Exchange
Natural's Ice Cream
Masala Library
Starbucks Coffee Capital
MCA Club
Smoke House Deli (BKC)
Phoenix Marketcity
The Irish House Phoenix Marketcity
212 All Day
Nilesh Dry Fruits
IVY Restaurant & Banquets
Yauatcha
Theobrama
Grand Hyatt
SodaBottleOpenerWala
Starbucks
Chembur Post Office Wada Pav
ZARA
Naaz Hotel
Khau Galli
ITC Maratha
Delhi Zaika
JW Marriott Mumbai Sahar
Botticino - Trident Hotel
24 Shisha Lounge
Gloria Jeans Coffee
Emirates First and Business Class Checkin
Glass House @ Hyatt Hotel
Peshawari
Starbucks
Maya
Poptate's
Darshan
Toast And Tonic
Sadguru Juice and Snacks
MCA Club Bar
the chocolate room
Pavilion
The Royal Orchid
The Good Wife
Cream Centre
Pooja Malai Gola
Lamba Da Dhaba
Natural Ice Cream
Club Lounge
Guru Kripa
Vikrant Circle
Hyatt Regency
J W Cafe
Astrix - The Lounge
Jhama Sweets
O Pedro
K 11 Fitness Gym & Academy
L5 Restaurant & Lounge
The Leela
Wok Exp

ValueError: Length mismatch: Expected axis has 7 elements, new values have 4 elements

In [ ]:
nearby_venues.groupby('name').count()

### One hot encoding

In [15]:
ch_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

ch_onehot['name'] = nearby_venues['name'] 

fixed_columns = [ch_onehot.columns[-1]] + list(ch_onehot.columns[:-1])
ch_onehot = ch_onehot[fixed_columns]

ch_onehot.head()

,name,Airport Service,American Restaurant,Asian Restaurant,Bagel Shop,Bar,Beer Garden,Café,Chinese Restaurant,Clothing Store,...,Punjabi Restaurant,Restaurant,Sculpture Garden,Shopping Mall,Snack Place,Spa,Sports Club,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant
0,Sofitel Mumbai BKC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hitchki,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PVR Cinemas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Starbucks Coffee: A Tata Alliance,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hamleys,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:

ch_grouped = ch_onehot.groupby('name').mean().reset_index()
ch_grouped

,name,Airport Service,American Restaurant,Asian Restaurant,Bagel Shop,Bar,Beer Garden,Café,Chinese Restaurant,Clothing Store,...,Punjabi Restaurant,Restaurant,Sculpture Garden,Shopping Mall,Snack Place,Spa,Sports Club,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant
0,212 All Day,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24 Shisha Lounge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Amreli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Astrix - The Lounge,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Botticino - Trident Hotel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Vikrant Circle,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
74,Wok Express BKC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,Yauatcha,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76,ZARA,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Sorting the venues for further analysis

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['name'] = ch_grouped['name']

for ind in np.arange(ch_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(ch_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,212 All Day,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
1,24 Shisha Lounge,Lounge,Vegetarian / Vegan Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store
2,Amreli,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store
3,Astrix - The Lounge,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
4,Botticino - Trident Hotel,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store


### CLUSTERING

In [19]:
kclusters = 5

ch_clustering = ch_grouped.drop('name', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ch_clustering)

kmeans.labels_[0:10]


array([0, 0, 0, 4, 0, 0, 1, 0, 0, 0], dtype=int32)

In [20]:
ch_merged = results

ch_merged = nearby_venues.join(venues_sorted.set_index('name'), on='name')

ch_merged.head()

,name,categories,lat,lng,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sofitel Mumbai BKC,Hotel,19.067448,72.869006,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store
1,Hitchki,Bar,19.069730,72.869761,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
2,PVR Cinemas,Movie Theater,19.086643,72.889839,Movie Theater,Vegetarian / Vegan Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store
3,Starbucks Coffee: A Tata Alliance,Coffee Shop,19.069457,72.869375,Coffee Shop,Vegetarian / Vegan Restaurant,Food Truck,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Clothing Store
4,Hamleys,Toy / Game Store,19.086655,72.889783,Toy / Game Store,Vegetarian / Vegan Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Dessert Shop,Deli / Bodega


### Conclusion

From the above analysis, it clearly shows few common places and most famous places of Mumbai. This list will definately help a tourist reach to famous stes and places.